In [1]:
import pandas as pd

## Institutes

In [2]:
df = pd.read_csv('../outputs/clean/institutes.csv')
df.head()

,id istituto,nome istituto,tipo istituto,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,dati aggiornati al,personale polizia penitenziaria aggiornato al,personale amministrativo aggiornato al,data di aggiornamento spazi detentivi
0,MII179988,Reggio Calabria Arghillà,Casa circondariale,294,1,326,135,164,12,32,...,85.0,0.0,3.0,85.0,85.0,85.0,2024-10-20,2024-09-30,2024-09-30,2024-10-20
1,MII172610,Brescia Verziano,Casa di reclusione,71,1,115,77,95,2,0,...,71.0,0.0,0.0,71.0,10.0,71.0,2024-10-20,2024-09-30,2024-09-30,2024-10-20
2,MII172320,Busto Arsizio,Casa circondariale,240,14,438,184,190,15,21,...,154.0,170.0,0.0,177.0,177.0,176.0,2024-10-20,2024-09-30,2024-09-30,2024-10-20
3,MII173712,Como,Casa circondariale,226,1,430,198,216,17,23,...,55.0,177.0,0.0,200.0,4.0,4.0,2024-10-20,2024-09-30,2024-09-30,2024-10-20
4,MII173747,Cremona,Casa circondariale,394,10,599,177,202,17,23,...,68.0,216.0,4.0,252.0,77.0,77.0,2024-10-20,2024-09-30,2024-09-30,2024-10-20


## Most recent data

In [3]:
# Get the most recent update
df['dati aggiornati al'] = pd.to_datetime(df['dati aggiornati al'], format='%Y-%m-%d')
df_most_recent = df.loc[df.groupby('id istituto')['dati aggiornati al'].idxmax()]

# Adding columns for places available and overcrowding index
df_most_recent['posti disponibili'] = df_most_recent['posti regolamentari'] - df_most_recent['posti non disponibili']
df_most_recent['tasso di affollamento'] = round(((df_most_recent['totale detenuti'] / df_most_recent['posti disponibili'])*100),0)

In [4]:
df_most_recent.value_counts(['dati aggiornati al'])

dati aggiornati al
2024-10-26            127
2024-10-25             63
Name: count, dtype: int64

In [5]:
df_most_recent.columns

Index(['id istituto', 'nome istituto', 'tipo istituto', 'posti regolamentari',
       'posti non disponibili', 'totale detenuti',
       'polizia penitenziaria - effettivi', 'polizia penitenziaria - previsti',
       'amministrativi - effettivi', 'amministrativi - previsti',
       'educatori - effettivi', 'educatori - previsti', 'numero complessivo',
       'numero non disponibili', 'doccia', 'bidet', 'portatori di handicap',
       'servizi igienici con porta', 'accensione luce autonoma',
       'prese elettriche', 'dati aggiornati al',
       'personale polizia penitenziaria aggiornato al',
       'personale amministrativo aggiornato al',
       'data di aggiornamento spazi detentivi', 'posti disponibili',
       'tasso di affollamento'],
      dtype='object')

In [6]:
# Adding institutes' information for mapping the institutes
# Read institutes' csv

df_info = pd.read_csv('../outputs/clean/institutes_info.csv')
df_info = df_info.rename(columns={'id_istituto': 'id istituto'})

merged_df = pd.merge(df_most_recent, df_info, on='id istituto')

# Adding additional columns
merged_df['posti disponibili'] = merged_df['posti regolamentari'] - merged_df['posti non disponibili']

merged_df['tasso di affollamento'] = round((merged_df['totale detenuti'] / merged_df['posti disponibili']) * 100, 0)


# Keep only relevant columns
df_filtered = merged_df[
  [
    'id istituto',
    'nome istituto',
    'tasso di affollamento',
    'indirizzo',
    'tipo istituto',
   'posti regolamentari',
    'posti non disponibili',
    'posti disponibili',
    'totale detenuti',
    'dati aggiornati al',
    'polizia penitenziaria - previsti',
    'polizia penitenziaria - effettivi',
    'personale polizia penitenziaria aggiornato al',
    'amministrativi - effettivi',
    'amministrativi - previsti',
    'personale amministrativo aggiornato al',
    'numero complessivo',
    'numero non disponibili',
    'doccia',
    'bidet',
    'portatori di handicap',
    'servizi igienici con porta',
    'accensione luce autonoma',
    'prese elettriche',
    'data di aggiornamento spazi detentivi',
    'latitudine',
    'longitude'
    ]
    ]

df_filtered.head()

,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,data di aggiornamento spazi detentivi,latitudine,longitude
0,MII152284,Genova Marassi,126.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,674,2024-10-26,...,0.0,128.0,175.0,3.0,213.0,100.0,0.0,2024-10-26,44.417913,8.951100
1,MII157783,Alba,84.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,41,2024-10-25,...,86.0,22.0,112.0,1.0,112.0,22.0,22.0,2024-10-25,44.691542,8.025636
2,MII158895,Milano San Vittore,229.0,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,702,240,462,1056,2024-10-26,...,182.0,272.0,184.0,0.0,327.0,325.0,325.0,2024-10-26,45.461598,9.166399
3,MII158901,Belluno,130.0,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,7,82,107,2024-10-26,...,6.0,20.0,20.0,0.0,50.0,40.0,40.0,2024-10-26,46.139738,12.219297
4,MII158910,Agrigento,143.0,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,9,274,391,2024-10-26,...,8.0,192.0,42.0,0.0,247.0,228.0,0.0,2024-10-26,37.319422,13.617396


In [7]:
df_filtered['scheda istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id istituto'] + '">Vai alla scheda istituto</a>'


df_filtered.head()

/tmp/ipykernel_14729/3401019820.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['scheda istituto'] = '<a href="https://www.giustizia.it/giustizia/page/it/dettaglio_scheda_istituto_penitenziario?s=' + df_filtered['id istituto'] + '">Vai alla scheda istituto</a>'


,id istituto,nome istituto,tasso di affollamento,indirizzo,tipo istituto,posti regolamentari,posti non disponibili,posti disponibili,totale detenuti,dati aggiornati al,...,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,data di aggiornamento spazi detentivi,latitudine,longitude,scheda istituto
0,MII152284,Genova Marassi,126.0,Piazzale Marassi n.2 - 16139 Genova,Casa circondariale,535,0,535,674,2024-10-26,...,128.0,175.0,3.0,213.0,100.0,0.0,2024-10-26,44.417913,8.951100,"<a href=""https://www.giustizia.it/giustizia/pa..."
1,MII157783,Alba,84.0,Loc.Toppino - Via Vivaro n. 14 - 12051 Alba,Casa di reclusione - Giuseppe Montalto,138,89,49,41,2024-10-25,...,22.0,112.0,1.0,112.0,22.0,22.0,2024-10-25,44.691542,8.025636,"<a href=""https://www.giustizia.it/giustizia/pa..."
2,MII158895,Milano San Vittore,229.0,Piazza Filangieri n.2 - 20123 Milano,Casa circondariale - Francesco Di Cataldo,702,240,462,1056,2024-10-26,...,272.0,184.0,0.0,327.0,325.0,325.0,2024-10-26,45.461598,9.166399,"<a href=""https://www.giustizia.it/giustizia/pa..."
3,MII158901,Belluno,130.0,Via Baldenich n.11 - 32100 Belluno,Casa circondariale,89,7,82,107,2024-10-26,...,20.0,20.0,0.0,50.0,40.0,40.0,2024-10-26,46.139738,12.219297,"<a href=""https://www.giustizia.it/giustizia/pa..."
4,MII158910,Agrigento,143.0,Piazza Di Lorenzo n. 1 - 92100 Agrigento,Casa circondariale - Pasquale Di Lorenzo,283,9,274,391,2024-10-26,...,192.0,42.0,0.0,247.0,228.0,0.0,2024-10-26,37.319422,13.617396,"<a href=""https://www.giustizia.it/giustizia/pa..."


In [8]:
# Saving csv
df_filtered.to_csv('../outputs/viz/institutes_most_recent.csv', index=False, encoding='UTF-8-sig')

## 1. Totals

In [9]:
grouped_df = df.groupby('dati aggiornati al').sum(numeric_only=True).reset_index()
grouped_df

,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,educatori - effettivi,educatori - previsti,numero complessivo,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-10-07,51196,4441,61846,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-10-09,51195,4439,61863,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-10-10,51195,4437,61843,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2024-10-11,51199,4441,61846,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2024-10-12,51199,4439,61885,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2024-10-13,51199,4439,61910,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2024-10-14,51199,4445,61844,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2024-10-17,50858,4458,61572,30890,33804,3402,4058,974,996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
grouped_df['posti disponibili'] = (grouped_df['posti regolamentari'] - grouped_df['posti non disponibili']).round(0)
grouped_df['tasso di affollamento'] = (grouped_df['totale detenuti'] / grouped_df['posti disponibili'] * 100).round(4).astype(float)

grouped_df.head()


,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,polizia penitenziaria - effettivi,polizia penitenziaria - previsti,amministrativi - effettivi,amministrativi - previsti,educatori - effettivi,educatori - previsti,numero complessivo,numero non disponibili,doccia,bidet,portatori di handicap,servizi igienici con porta,accensione luce autonoma,prese elettriche,posti disponibili,tasso di affollamento
0,2024-10-05,51196,4384,61880,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46812,132.1883
1,2024-10-06,51196,4384,61912,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46812,132.2567
2,2024-10-07,51196,4441,61846,31433,34035,3388,4049,969,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46755,132.2768
3,2024-10-09,51195,4439,61863,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46756,132.3103
4,2024-10-10,51195,4437,61843,31091,34035,3414,4079,978,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46758,132.2619


In [11]:
grouped_df = grouped_df[['dati aggiornati al', 'posti regolamentari', 'posti non disponibili', 'totale detenuti', 'tasso di affollamento']]
# grouped_df['posti disponibili'] = grouped_df['posti regolamentari'] - grouped_df['posti non disponibili']
# grouped_df['tasso_affollamento'] = round((grouped_df['posti_occupati'] / grouped_df['posti_disponibili'])*100,4).astype(float)
grouped_df.head()


,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,tasso di affollamento
0,2024-10-05,51196,4384,61880,132.1883
1,2024-10-06,51196,4384,61912,132.2567
2,2024-10-07,51196,4441,61846,132.2768
3,2024-10-09,51195,4439,61863,132.3103
4,2024-10-10,51195,4437,61843,132.2619


In [12]:
grouped_df.to_csv('../outputs/viz/institutes_totals.csv', index=False)

## Personale

In [13]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')
df.columns

Index(['id istituto', 'nome istituto', 'tasso di affollamento', 'indirizzo',
       'tipo istituto', 'posti regolamentari', 'posti non disponibili',
       'posti disponibili', 'totale detenuti', 'dati aggiornati al',
       'polizia penitenziaria - previsti', 'polizia penitenziaria - effettivi',
       'personale polizia penitenziaria aggiornato al',
       'amministrativi - effettivi', 'amministrativi - previsti',
       'personale amministrativo aggiornato al', 'numero complessivo',
       'numero non disponibili', 'doccia', 'bidet', 'portatori di handicap',
       'servizi igienici con porta', 'accensione luce autonoma',
       'prese elettriche', 'data di aggiornamento spazi detentivi',
       'latitudine', 'longitude', 'scheda istituto'],
      dtype='object')

In [14]:
df = pd.read_csv('../outputs/viz/institutes_most_recent.csv')


df_polizia = df[['nome istituto', 'totale detenuti', 'tasso di affollamento', 'polizia penitenziaria - previsti', 'polizia penitenziaria - effettivi', 'personale polizia penitenziaria aggiornato al', 'dati aggiornati al']]

df_polizia['polizia penitenziaria - mancante'] = df_polizia['polizia penitenziaria - previsti'] - df_polizia['polizia penitenziaria - effettivi']

df_polizia.head(2)

/tmp/ipykernel_14729/2362681573.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['polizia penitenziaria - mancante'] = df_polizia['polizia penitenziaria - previsti'] - df_polizia['polizia penitenziaria - effettivi']


,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante
0,Genova Marassi,674,126.0,336,336,2024-09-30,2024-10-26,0
1,Alba,41,84.0,107,88,2024-09-30,2024-10-25,19


In [15]:
df_polizia['polizia penitenziaria - mancante percentuale'] = round(df_polizia['polizia penitenziaria - mancante'] / df_polizia['polizia penitenziaria - previsti']*100,2)
df_polizia.head(2)


/tmp/ipykernel_14729/989766878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_polizia['polizia penitenziaria - mancante percentuale'] = round(df_polizia['polizia penitenziaria - mancante'] / df_polizia['polizia penitenziaria - previsti']*100,2)


,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
0,Genova Marassi,674,126.0,336,336,2024-09-30,2024-10-26,0,0.00
1,Alba,41,84.0,107,88,2024-09-30,2024-10-25,19,17.76


In [16]:
df_polizia_clean = df_polizia[df_polizia['totale detenuti'] != 0]
df_polizia_clean.sort_values('polizia penitenziaria - mancante percentuale', ascending=False).head(2)

,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
136,Roma Rebibbia IIIª casa,81,61.0,46,31,2024-09-30,2024-10-26,15,32.61
158,Tempio Pausania,173,102.0,146,104,2024-09-30,2024-10-26,42,28.77


In [17]:
critical_prisons = df_polizia_clean[(df_polizia_clean['tasso di affollamento'] > 120) & (df_polizia_clean['polizia penitenziaria - mancante percentuale'] > 20)].reset_index(drop=True)
critical_prisons

,nome istituto,totale detenuti,tasso di affollamento,polizia penitenziaria - previsti,polizia penitenziaria - effettivi,personale polizia penitenziaria aggiornato al,dati aggiornati al,polizia penitenziaria - mancante,polizia penitenziaria - mancante percentuale
0,Ancona Montacuto,338,132.0,175,125,2024-09-30,2024-10-26,50,28.57
1,Augusta,557,163.0,224,175,2024-09-30,2024-10-26,49,21.88
2,Rieti,507,187.0,175,137,2024-09-30,2024-10-25,38,21.71
3,Bolzano,107,126.0,77,61,2024-09-30,2024-10-25,16,20.78
4,Cassino,209,165.0,142,107,2024-09-30,2024-10-25,35,24.65
5,Lucca,81,198.0,91,71,2024-09-30,2024-10-26,20,21.98
6,Matera,190,151.0,125,97,2024-09-30,2024-10-26,28,22.40
7,Pescara,456,168.0,166,122,2024-09-30,2024-10-26,44,26.51
8,Roma Regina Coeli,1098,193.0,480,347,2024-09-30,2024-10-25,133,27.71
9,Rossano,318,121.0,139,109,2024-09-30,2024-10-25,30,21.58


In [18]:
critical_prisons.to_csv('../outputs/viz/institutes_critical.csv', index=False, encoding='UTF-8-sig')

### Tasso Reale

In [19]:
df1 = pd.read_csv('../outputs/viz/bulletines_totals.csv')
df2 = pd.read_csv('../outputs/viz/institutes_totals.csv')

In [20]:
df2.tail(2)

,dati aggiornati al,posti regolamentari,posti non disponibili,totale detenuti,tasso di affollamento
13,2024-10-25,64999,5672,78505,132.3259
14,2024-10-26,37332,3346,45365,133.4814


In [21]:
# Renaming columns for clarity and merging on a unified date column
df1.rename(columns={'Ultimo aggiornamento': 'Date', 'tasso_affollamento': 'tasso_affollamento_ufficiale'}, inplace=True)
df2.rename(columns={'dati aggiornati al': 'Date', 'tasso di affollamento': 'tasso_affollamento_reale'}, inplace=True)

In [22]:
# Converting the Date columns to datetime for consistency
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

In [23]:
df2.tail(2)

,Date,posti regolamentari,posti non disponibili,totale detenuti,tasso_affollamento_reale
13,2024-10-25,64999,5672,78505,132.3259
14,2024-10-26,37332,3346,45365,133.4814


In [24]:
merged_df = pd.merge(df1[['Date', 'tasso_affollamento_ufficiale']], 
                     df2[['Date', 'tasso_affollamento_reale']], 
                     on='Date', 
                     how='outer')
merged_df.sort_values(by='Date', inplace=True)

merged_df.tail(2)

,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
82,2024-10-25,NaN,132.3259
83,2024-10-26,NaN,133.4814


In [25]:
filtered_df = merged_df[merged_df['Date'] > '2024-08-01']
filtered_df.tail(2)


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale
82,2024-10-25,NaN,132.3259
83,2024-10-26,NaN,133.4814


In [26]:
# Apply linear interpolation for missing values
filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpolate(method='linear')),4)

filtered_df.tail(2)


/tmp/ipykernel_14729/3090938612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tasso_affollamento_ufficiale (interpolated)'] = round((filtered_df['tasso_affollamento_ufficiale'].interpolate(method='linear')),4)
/tmp/ipykernel_14729/3090938612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tasso_affollamento_reale (interpolated)'] = round((filtered_df['tasso_affollamento_reale'].interpolate(method='linear')),4)


,Date,tasso_affollamento_ufficiale,tasso_affollamento_reale,tasso_affollamento_ufficiale (interpolated),tasso_affollamento_reale (interpolated)
82,2024-10-25,NaN,132.3259,120.8337,132.3259
83,2024-10-26,NaN,133.4814,120.8337,133.4814


In [27]:
filtered_df.to_csv('../outputs/viz/tasso_affollamento.csv', index=False)